## Dataset

Before beginning the analysis, we will load and view the dataset, and perform some initial cleaning.

* View the dataset info:

In [ ]:
# load libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# load dataset
df = pd.read_csv('../input/marketing-data/marketing_data.csv')
print(df.info())
df.head()


* Transform selected columns to numeric format:
    - `Income` to float

In [ ]:
# clean up column names that contain whitespace
df.columns = df.columns.str.replace(' ', '')

# transform Income column to a numerical
df['Income'] = df['Income'].str.replace('$', '')
df['Income'] = df['Income'].str.replace(',', '').astype('float')

* The cleaned dataset:

In [ ]:
df.head()

# Section 01: Exploratory Data Analysis

* Identify features containing null values:

In [ ]:
# null values
df.isnull().sum().sort_values(ascending=False)

* The feature `Income` contains 24 null values. Plotting to identify best strategy for imputation

In [ ]:
plt.figure(figsize=(8,4))
sns.distplot(df['Income'], kde=False, hist=True)
plt.title('Income distribution', size=16)
plt.ylabel('count');

 - Findings: 
        - Most incomes are distributed between $0-$100,000, with a few outliers
        - Will impute null values with median value, to avoid effects of outliers on imputation value

In [ ]:
df['Income'].plot(kind='box', figsize=(3,4), patch_artist=True)

* Imputing null values in `Income`, using median value (to avoid skewing of the mean due to outliers):

In [ ]:
df['Income'] = df['Income'].fillna(df['Income'].median())

#### Outliers

* Identify features containing outliers

In [ ]:
# select columns to plot
df_to_plot = df.drop(columns=['ID', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response', 'Complain']).select_dtypes(include=np.number)

# subplots
df_to_plot.plot(subplots=True, layout=(4,4), kind='box', figsize=(12,14), patch_artist=True)
plt.subplots_adjust(wspace=0.5);

- Findings: Multiple features contain outliers, but the only that likely indicate data entry errors are `Year_Birth <= 1900`

* Removing rows where `Year_Birth <= 1900`:

In [ ]:
df = df[df['Year_Birth'] > 1900].reset_index(drop=True)

plt.figure(figsize=(3,4))
df['Year_Birth'].plot(kind='box', patch_artist=True);

In [ ]:
df.info()

* Transform `Dt_Customer` to datetime:

In [ ]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])

    - The total number of dependents in the home ('Dependents') can be obtained from the sum of 'Kidhome' and 'Teenhome'
    - The year of becoming a customer ('Year_Customer') can be obtained from 'Dt_Customer'
    - The total amount spent ('TotalMnt') can be obtained from the sum of all features containing the keyword 'Mnt'
    - The total purchases ('TotalPurchases') can be obtained from the sum of all features containing the keyword 'Purchases'
    - The total number of campains accepted ('TotalCampaignsAcc') can be obtained from the sum of all features containing the keywords 'Cmp' and 'Response' (the latest campaign)

In [ ]:
list(df.columns)

* Perform feature engingeering as outlined in notes above:

In [ ]:
# Dependents
df['Dependents'] = df['Kidhome'] + df['Teenhome']

# Year becoming a Customer
df['Year_Customer'] = pd.DatetimeIndex(df['Dt_Customer']).year

# Total Amount Spent
mnt_cols = [col for col in df.columns if 'Mnt' in col]
df['TotalMnt'] = df[mnt_cols].sum(axis=1)

# Total Purchases
purchases_cols = [col for col in df.columns if 'Purchases' in col]
df['TotalPurchases'] = df[purchases_cols].sum(axis=1)

# Total Campaigns Accepted
campaigns_cols = [col for col in df.columns if 'Cmp' in col] + ['Response'] # 'Response' is for the latest campaign
df['TotalCampaignsAcc'] = df[campaigns_cols].sum(axis=1)

# view new features, by customer ID
df[['ID', 'Dependents', 'Year_Customer', 'TotalMnt', 'TotalPurchases', 'TotalCampaignsAcc']].head()

* To identify patterns, we will first identify feature correlations. Positive correlations between features appear red, negative correlations appear blue, and no correlation appears grey in the clustered heatmap below.

In [ ]:
# calculate correlation matrix
df1=df.drop(columns='ID')
corr = df1.corr()
corr.style.background_gradient(cmap='RdYlGn')

* From this heatmap we can observe the following clusters of correlated features:
        - Amount spent and number of purchases are positively correlated with 'Income'
        - Purchasing in store, on the web, or via the catalog is positively correlated with 'Income'
        - Amount spent and number of purchases are negatively correlated with 'Dependents' (with a stronger effect from kids *vs.* teens)
        - Purchasing deals is positively correlated with 'Dependents' (kids and/or teens) and negatively correlated with 'Income'
        - Acceptance of the advertising campaigns ('AcceptedCmp' and 'Response') are strongly positively correlated with each other
        - Weak positive correlation of the advertising campaigns is seen with the "High Income" cluster, and weak negative correlation is seen with the "Have Kids & Teens" cluster
        
        * Anomalies:
    - Surprisingly, the number of website visits in the last month does not correlate with an increased number of web purchases
    - Instead, 'NumWebVisitsMonth' is positively correlated with the number of deals purchased, suggesting that  suggesting that deals are an effective way of stimulating purchases on the website

* Plot illustrating the effect of high income on spending

In [ ]:
sns.lmplot(x='Income', y='TotalMnt', data=df[df['Income'] < 200000]);

* Plot illustrating negative effect of having dependents (kids & teens) on spending:

In [ ]:
plt.figure(figsize=(4,4))
sns.boxplot(x='Dependents', y='TotalMnt', data=df);

* Plot illustrating positive effect of having dependents (kids & teens) on number of deals purchased:

In [ ]:
plt.figure(figsize=(4,4))
sns.boxplot(x='Dependents', y='NumDealsPurchases', data=df);

* Plots illustrating the positive effect of income and negative effect of having kids & teens on advertising campaign acceptance:

Note: For the purposes of the following plot, limiting income to < 200000 to remove outlier

In [ ]:
plt.figure(figsize=(5.5,4))
sns.boxplot(x='TotalCampaignsAcc', y='Income', data=df[df['Income']<200000]);

In [ ]:
plt.figure(figsize=(5.5,4))
sns.boxplot(x='TotalCampaignsAcc', y='Dependents', data=df);

* Investigate anomaly: 
    - Number of web visits in the last month is not positively correlated with number of web purchases
    - Instead, it is positively correlated with the number of deals purchased, suggesting that deals are an effective way of stimulating purchases on the website

In [ ]:
sns.lmplot(x='NumWebVisitsMonth', y='NumWebPurchases', data=df);

In [ ]:
sns.lmplot(x='NumWebVisitsMonth', y='NumDealsPurchases', data=df);

# Statistical Analysis


In [ ]:
plt.figure(figsize=(8,3))
sns.distplot(df['NumStorePurchases'], kde=False, hist=True, bins=12)
plt.title('NumStorePurchases distribution', size=16)
plt.ylabel('count');

* Drop uninformative features
    - `ID` is unique to each customer
    - `Dt_Customer` will be dropped in favor of using engineered variable `Year_Customer`
* Perform one-hot encoding of categorical features, encoded data shown below:

In [ ]:
# drop unique ID
df.drop(columns=['ID', 'Dt_Customer'], inplace=True)

In [ ]:
# one-hot encoding of categorical features
from sklearn.preprocessing import OneHotEncoder

# get categorical features and review number of unique values
cat = df.select_dtypes(exclude=np.number)
print("Number of unique values per categorical feature:\n", cat.nunique())

# use one hot encoder
enc = OneHotEncoder(sparse=False).fit(cat)
cat_encoded = pd.DataFrame(enc.transform(cat))
cat_encoded.columns = enc.get_feature_names(cat.columns)

# merge with numeric data
num = df.drop(columns=cat.columns)
df2 = pd.concat([cat_encoded, num], axis=1)
df2.head()

* Fit linear regression model to training data (70% of dataset)
* Evaluate predictions on test data (30% of dataset) using RMSE:
    - Findings: The RMSE is exceedingly small compared to the median value of the target variable, indicating good model predictions

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# isolate X and y variables, and perform train-test split
X = df2.drop(columns='NumStorePurchases')
y = df2['NumStorePurchases']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# predictions
preds = model.predict(X_test)

# evaluate model using RMSE
print("Linear regression model RMSE: ", np.sqrt(mean_squared_error(y_test, preds)))
print("Median value of target variable: ", y.median())

* Identify features that significantly affect the number of store purchases, using permutation importance:
    - Significant features:
        - 'TotalPurchases', 'NumCatalogPurchases', 'NumWebPurchases', 'NumDealsPurchases'
        - All other features are not significant

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(model, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist(), top=5)

* Explore the directionality of these effects, using SHAP values:
    - Findings:
        - The number of store purchases increases with higher number of total purchases ('TotalPurchases')
        - The number of store purchases decreases with higher number of catalog, web, or deals purchases ('NumCatalogPurchases', 'NumWebPurchases', 'NumDealsPurchases')
    - Interpretation:
        - Customers who shop the most in stores are those who shop less via the catalog, website, or special deals

In [ ]:
import shap

# calculate shap values 
ex = shap.Explainer(model, X_train)
shap_values = ex(X_test)

# plot
plt.title('SHAP summary for NumStorePurchases', size=16)
shap.plots.beeswarm(shap_values, max_display=5);

- Findings: 
        - Spain (SP) has the highest number of purchases
        - US is second to last, therefore the US does not fare better than the rest of the world in terms of the total number of purchases

In [ ]:
plt.figure(figsize=(5,4))
df.groupby('Country')['TotalPurchases'].sum().sort_values(ascending=False).plot(kind='bar')
plt.title('Total Number of Purchases by Country', size=16)
plt.ylabel('Number of Purchases');

* Plot total amount spent by country: 
    - Findings: 
        - Spain (SP) has the highest total amount spent on purchases
        - US is second to last, therefore the US does not fare better than the rest of the world in terms of the total amount spent on purchases

In [ ]:
plt.figure(figsize=(5,4))
df.groupby('Country')['TotalMnt'].sum().sort_values(ascending=False).plot(kind='bar')
plt.title('Total Amount Spent by Country', size=16)
plt.ylabel('Amount Spent');

- Findings: There is a positive relationship, but is it statistically significant?

In [ ]:
sns.lmplot(x='MntGoldProds', y='NumStorePurchases', data = df);

- Findings: There is significant positive correlation between `MntGoldProds` and `NumStorePurchases`

In [ ]:
from scipy.stats import kendalltau

kendall_corr = kendalltau(x=df['MntGoldProds'], y=df['NumStorePurchases'])

# print results
print('Kendall correlation (tau): ', kendall_corr.correlation)
print('Kendall p-value: ', kendall_corr.pvalue)

- Findings: Married PhD candidates spend significantly less on fish products compared to other customers.

In [ ]:
# sum the marital status and phd dummy variables - the Married+PhD group will have value of 2
df2['Married_PhD'] = df2['Marital_Status_Married'] + df2['Education_PhD']
df2['Married_PhD'] = df2['Married_PhD'].replace({2:'Married-PhD', 1:'Other', 0:'Other'})

# plot MntFishProducts between Married-PhD and others
plt.figure(figsize=(2.5,4))
sns.boxplot(x='Married_PhD', y='MntFishProducts', data=df2);

In [ ]:
# independent t-test p-value
from scipy.stats import ttest_ind
pval = ttest_ind(df2[df2['Married_PhD'] == 'Married-PhD']['MntFishProducts'], df2[df2['Married_PhD'] == 'Other']['MntFishProducts']).pvalue
print("t-test p-value: ", round(pval, 3))

In [ ]:
# now drop the married-phD column created above, to include only the original variables in the analysis below
df2.drop(columns='Married_PhD', inplace=True)

In [ ]:
plt.figure(figsize=(8,3))
sns.distplot(df['MntFishProducts'], kde=False, hist=True, bins=12)
plt.title('MntFishProducts distribution', size=16)
plt.ylabel('count');

- Findings: The RMSE is exceedingly small compared to the median value of the target variable, indicating good model predictions

In [ ]:
# isolate X and y variables, and perform train-test split
X = df2.drop(columns='MntFishProducts')
y = df2['MntFishProducts']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# predictions
preds = model.predict(X_test)

# evaluate model using RMSE
print("Linear regression model RMSE: ", np.sqrt(mean_squared_error(y_test, preds)))
print("Median value of target variable: ", y.median())

- Significant features:
        - 'TotalMnt', 'MntWines', 'MntMeatProducts', 'MntGoldProds', 'MntSweetProducts', 'MntFruits'
        - All other features are not significant

In [ ]:
perm = PermutationImportance(model, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist(), top=7)

    - Findings:
        - The amount spent on fish increases with higher total amount spent ('TotalMnt')
        - The amount spent on fish decreases with higher amounts spent on wine, meat, gold, fruit, or sweets ('MntWines', 'MntMeatProducts', 'MntGoldProds', 'MntSweetProducts', 'MntFruits')
    - Interpretation:
        - Customers who spend the most on fish are those who spend less on other products (wine, meat, gold, fruit, and sweets)

In [ ]:
import shap

# calculate shap values 
ex = shap.Explainer(model, X_train)
shap_values = ex(X_test)

# plot
plt.title('SHAP summary for MntFishProducts', size=16)
shap.plots.beeswarm(shap_values, max_display=7);

    - Findings:
        - The campaign acceptance rates are low overall
        - The campaign with the highest overall acceptance rate is the most recent campaign (column name: `Response`)
        - The country with the highest acceptance rate in any campaign is Mexico
    - Is the effect of region on campaign success statistically significant? See below.

In [ ]:
# convert country codes to correct nomenclature for choropleth plot
# the dataset doesn't provide information about country codes
## ...so I'm taking my best guess about the largest nations that make sense given the codes provided
df['Country_code'] = df['Country'].replace({'SP': 'ESP', 'CA': 'CAN', 'US': 'USA', 'SA': 'ZAF', 'ME': 'MEX'})

# success of campaigns by country code
df_cam = df[['Country_code', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response']].melt(
    id_vars='Country_code', var_name='Campaign', value_name='Accepted (%)')
df_cam = pd.DataFrame(df_cam.groupby(['Country_code', 'Campaign'])['Accepted (%)'].mean()*100).reset_index(drop=False)

# rename the campaign variables so they're easier to interpret
df_cam['Campaign'] = df_cam['Campaign'].replace({'AcceptedCmp1': '1',
                                                'AcceptedCmp2': '2',
                                                'AcceptedCmp3': '3',
                                                'AcceptedCmp4': '4',
                                                'AcceptedCmp5': '5',
                                                 'Response': 'Most recent'
                                                })

# choropleth plot
import plotly.express as px

fig = px.choropleth(df_cam, locationmode='ISO-3', color='Accepted (%)', facet_col='Campaign', facet_col_wrap=2,
                    facet_row_spacing=0.05, facet_col_spacing=0.01, width=700,
                    locations='Country_code', projection='natural earth', title='Advertising Campaign Success Rate by Country'
                   )
fig.show()

* Statistical summary of regional effects on campaign success:
    - Methodology: Performed logistic regression for Campaign Accepted by Country, reporting Chisq p-value for overall model.
    - Findings: The regional differences in advertising campaign success are statistically significant.

In [ ]:
# calculate logistic regression p-values for campaign acceptance ~ country using generalized linear model
import statsmodels.formula.api as smf
import statsmodels as sm
from scipy import stats

## get the data of interest for glm
df_cam_wide = df[['Country', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response']]

## to store statistics results
stat_results = []

## perform glm
for col in df_cam_wide.drop(columns='Country').columns:
    this_data = df_cam_wide[['Country', col]]
    
    # define formula
    formula = col+'~Country'
    
    # logistic regression (family=binomial)
    model = smf.glm(formula = formula, data=this_data, family=sm.genmod.families.Binomial())
    result = model.fit()
    
    # get chisquare value for overall model (CampaignAccepted ~ Country) and calculate p-value
    chisq = result.pearson_chi2
    pval = stats.distributions.chi2.sf(chisq , 7) # Df Model = 7 degrees of freedom when you run result.summary()
     
    # append to stat_results
    stat_results.append(pval)
    
    # print stat summary for entire model
    print(result.summary())
    
## check results
print("\nChisq p-values: ", stat_results)

In [ ]:
# plotting
## merge in the original country codes provided in the dataset
countries = df[['Country', 'Country_code']].drop_duplicates().reset_index(drop=True)
df_cam2 = df_cam.merge(countries, how='left', on='Country_code')
df_cam2.head()

## bar graphs
g = sns.FacetGrid(df_cam2, col='Campaign', col_wrap=3)
g.map(sns.barplot, 'Country', 'Accepted (%)')
for ax, pval in zip(g.axes.flat, stat_results):
    ax.text(0, 65, "Chisq p-value: "+str(pval), fontsize=9) #add text;

# Data Visualization

In [ ]:
# calculate success rate (percent accepted)
cam_success = pd.DataFrame(df[['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response']].mean()*100, 
                           columns=['Percent']).reset_index()

# plot
sns.barplot(x='Percent', y='index', data=cam_success.sort_values('Percent'), palette='Blues')
plt.xlabel('Accepted (%)')
plt.ylabel('Campaign')
plt.title('Marketing campaign success rate', size=16);

### What does the average customer look like for this company?

* Basic demographics: The average customer is...
    - Born in 1969
    - Became a customer in 2013
    - Has an income of roughly \$52,000 per year
    - Has 1 dependent (roughly equally split between kids or teens)
    - Made a purchase from our company in the last 49 days

In [ ]:
# list of cols with binary responses
binary_cols = [col for col in df.columns if 'Accepted' in col] + ['Response', 'Complain']

# list of cols for spending 
mnt_cols = [col for col in df.columns if 'Mnt' in col]

# list of cols for channels
channel_cols = [col for col in df.columns if 'Num' in col] + ['TotalPurchases', 'TotalCampaignsAcc']

In [ ]:
# average customer demographics
demographics = pd.DataFrame(round(df.drop(columns=binary_cols+mnt_cols+channel_cols).mean(), 1), columns=['Average']).reindex([
    'Year_Birth', 'Year_Customer', 'Income', 'Dependents', 'Kidhome', 'Teenhome', 'Recency'])

demographics

### Which products are performing best?

* The average customer spent...
    - \$25-50 on Fruits, Sweets, Fish, or Gold products
    - Over \$160 on Meat products
    - Over \$300 on Wines
    - Over \$600 total
* Products performing best:
    - Wines
    - Followed by meats

In [ ]:
spending = pd.DataFrame(round(df[mnt_cols].mean(), 1), columns=['Average']).sort_values(by='Average').reset_index()

# plot
ax = sns.barplot(x='Average', y='index', data=spending, palette='Blues')
plt.ylabel('Amount spent on...')

## add text labels for each bar's value
for p,q in zip(ax.patches, spending['Average']):
    ax.text(x=q+40,
            y=p.get_y()+0.5,
            s=q,
            ha="center") ;

### Which channels are underperforming?

* Channels: The average customer...
    - Accepted less than 1 advertising campaign
    - Made 2 deals purchases, 2 catalog purchases, 4 web purchases, and 5 store purchases
    - Averaged 14 total purchases
    - Visited the website 5 times
* Underperforming channels:
    - Advertising campaigns
    - Followed by deals, and catalog

In [ ]:
channels = pd.DataFrame(round(df[channel_cols].mean(), 1), columns=['Average']).sort_values(by='Average').reset_index()

# plot
ax = sns.barplot(x='Average', y='index', data=channels, palette='Blues')
plt.ylabel('Number of...')

## add text labels for each bar's value
for p,q in zip(ax.patches, channels['Average']):
    ax.text(x=q+0.8,
            y=p.get_y()+0.5,
            s=q,
            ha="center") ;

# Conclusion

**Summary of actionable findings to improve advertising campaign success:**  
* The most successful advertising campaign was the most recent campaign (column name: `Response`), and was particularly successful in Mexico (>60% acceptance rate!)
    - Suggested action: Conduct future advertising campaigns using the same model recently implemented in Mexico. 
* Advertising campaign acceptance is positively correlated with income and negatively correlated with having kids/teens
    - Suggested action: Create two streams of targeted advertising campaigns, one aimed at high-income individuals without kids/teens and another aimed at lower-income individuals with kids/teens
* The most successful products are wines and meats (*i.e.* the average customer spent the most on these items)
    - Suggested action: Focus advertising campaigns on boosting sales of the less popular items
* The underperforming channels are deals and catalog purchases (*i.e.* the average customer made the fewest purchases via these channels)
* The best performing channels are web and store purchases (*i.e.* the average customer made the most purchases via these channels)
    - Suggested action: Focus advertising campaigns on the more successful channels, to reach more customers